# mlflow

In [1]:
import mlflow
import mlflow.lightgbm
import contextlib
import ipykernel
from notebook import notebookapp
import urllib
import json

In [2]:
def get_notebook_path():
    '''
    mlflow に notebook をそのまま保存する際に notebook のパスを取得するのに使います。
    単純にファイル名が取れないためかなりややこしいことになっています。
    使用するには jupyterlab が token ありで起動していないといけません。
    '''
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[1].split('.')[0]

    for server in notebookapp.list_running_servers():
        try:
            if server['token'] == '' and not server['password']:  # No token and no password, ahem...
                req = urllib.request.urlopen(server['url'] + 'api/sessions')
            elif server['token'] != '':
                req = urllib.request.urlopen(server['url'] + 'api/sessions?token=' + server['token'])
            else:
                continue
            sessions = json.load(req)
            for sess in sessions:
                if sess['kernel']['id'] == kernel_id:
                    return os.path.join(server['notebook_dir'], sess['notebook']['path'])
        except Exception:
            raise
    raise EnvironmentError('JupyterLab の token を指定するか、パスワードを無効化してください。')

In [3]:
class MLLogger:
    ...

    @contextlib.contextmanager
    def start(self, experiment_name: str, run_name: str) -> None:
        mlflow.set_experiment(experiment_name)
        with mlflow.start_run(run_name=run_name):
#             self.log_running()
#             self.log_user(self._user)
#             self.log_notebook()  # <- ここで notebook をアップロード
            try:
                yield
                self.log_notebook()  # <- ここで notebook をアップロード
#                 mlflow.log_success()
            except (Exception, KeyboardInterrupt):
#                 mlflow.log_failure()
                raise
    ...


In [4]:
def log_notebook():
    '''
    notebook を mlflow に保存します。
    '''
    notebook = get_notebook_path()  # notebook のパスを取得
    mlflow.log_artifact(notebook)  # mlflow.log_artifact(file) を呼んでるだけ

# Import

In [5]:
# ------------------------------------------------------------------------------
# Import libraries
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob

from sklearn.model_selection import KFold, GroupKFold
import lightgbm as lgb

import psutil
import random
import os
import time
import sys
import math
from contextlib import contextmanager
import glob


# Setting

In [6]:
# ------------------------------------------------------------------------------
# File path definition
# ------------------------------------------------------------------------------
DATA_DIR = Path("/home/knikaido/work/Indoor-Location-Navigation/data/")
WIFI_DIR = DATA_DIR / 'indoor-navigation-and-location-wifi-features'
MLFLOW_DIR = DATA_DIR / 'mlflow/mlruns'
OUTPUT_DIR = Path('./output/')

LOG_PATH = OUTPUT_DIR / "log"
LOG_PATH.mkdir(parents=True, exist_ok=True)

In [7]:
floor_dict = {'B1':-1, 'F1':0, 'F2':1 , 'F3':2, 'F4':3, 'F5':4, 'F6':5, 'F7':6, 'B2':-2, '1F':0, '2F':1,
       '3F':2, '4F':3, '5F':4, '6F':5, '7F':6, '8F':7, '9F':8, 'F8':7}

In [8]:
# ------------------------------------------------------------------------------
# Fixed values
# ------------------------------------------------------------------------------
N_SPLITS = 5
SEED = 777
EXP_NUM = 4
IS_SAVE = True

In [9]:
# ------------------------------------------------------------------------------
# Define parameters for models
# ------------------------------------------------------------------------------
lgb_params = {'objective': 'root_mean_squared_error',
              'boosting_type': 'gbdt',
              'n_estimators': 50000,
              'learning_rate': 0.05,
              'num_leaves': 90,
              'colsample_bytree': 0.4,
              'subsample': 0.6,
              'subsample_freq': 2,
              'bagging_seed': SEED,
              'reg_alpha': 8,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }

lgb_f_params = {'objective': 'multiclass',
                'boosting_type': 'gbdt',
                'n_estimators': 50000,
                'learning_rate': 0.1,
                'num_leaves': 90,
                'colsample_bytree': 0.4,
                'subsample': 0.6,
                'subsample_freq': 2,
                'bagging_seed': SEED,
                'reg_alpha': 10,
                'reg_lambda': 2,
                'random_state': SEED,
                'n_jobs': -1
                }


In [10]:
# ------------------------------------------------------------------------------
# Utilities
# ------------------------------------------------------------------------------
@contextmanager
def timer(name: str):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    try:
        yield
    finally:
        m1 = p.memory_info()[0] / 2. ** 30
        delta = m1 - m0
        sign = '+' if delta >= 0 else '-'
        delta = math.fabs(delta)
        print(f"[{m1:.1f}GB({sign}{delta:.1f}GB): {time.time() - t0:.3f}sec] {name}", file=sys.stderr)


def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

    
def comp_metric(xhat, yhat, fhat, x, y, f):
    pos_loss = np.sum(np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) / xhat.shape[0])
    floor_loss = np.sum(15 * np.abs(fhat-f) / xhat.shape[0])
    print(f'pos_loss = {pos_loss}, floor_loss = {floor_loss}')
    intermediate = pos_loss + floor_loss
    return intermediate


def score_log(df: pd.DataFrame, num_files: int, nam_file: str, data_shape: tuple, n_fold: int, seed: int, mpe: float):
    score_dict = {'n_files': num_files, 'file_name': nam_file, 'shape': data_shape, 'fold': n_fold, 'seed': seed, 'score': mpe}
    # noinspection PyTypeChecker
    df = pd.concat([df, pd.DataFrame.from_dict([score_dict])])
    df.to_csv(LOG_PATH / f"log_score.csv", index=False)
    return df


In [11]:
# wandb.init(project='Indoor_Location_Navigation', entity='sqrt4kaido', name=str(f'exp{EXP_NUM}'))
# wandb.config.pos_param = lgb_params
# wandb.config.f_param = lgb_f_params
set_seed(SEED)

# ------------------------------------------------------------------------------
# Read data
# ------------------------------------------------------------------------------
# feature_dir = "../input/indoor-navigation-and-location-wifi-features"
train_files = sorted(glob.glob(str(WIFI_DIR / '*_train.csv')))
test_files = sorted(glob.glob(str(WIFI_DIR / '*_test.csv')))
subm = pd.read_csv(str(DATA_DIR / 'indoor-location-navigation/sample_submission.csv'), index_col=0)

In [12]:
# oof_floor = pd.read_csv('../01/oof_preds.csv')
# oof_floor["path"] = oof_floor["path"].str.replace(".txt", "")
# oof_floor

In [13]:
# sub_floor = pd.read_csv('../01/sub1.csv')
# sub_floor

In [14]:
if IS_SAVE == True:

    mlflow.set_tracking_uri(str(MLFLOW_DIR))
    mlflow.start_run(run_name=str(f'exp{EXP_NUM}'))
    log_notebook()

    for k,v in lgb_params.items():
        mlflow.log_param('pos_'+k,v)

    for k,v in lgb_f_params.items():
        mlflow.log_param('floor_'+k,v)
    mlflow.lightgbm.autolog()

2021/03/25 01:51:29 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



# Train

In [ ]:
# ------------------------------------------------------------------------------
# Training and inference
# ------------------------------------------------------------------------------
score_df = pd.DataFrame()
oof = list()
predictions = list()
for n_files, file in enumerate(train_files):
    data = pd.read_csv(file, index_col=0)
    test_data = pd.read_csv(test_files[n_files], index_col=0)
    
#     floor_pred = data.merge(oof_floor, how='left', on='path')['floor_pred'].map(floor_dict)
#     data.insert(0, 'floor_pred', floor_pred)

#     test_floor_pred = test_data.merge(sub_floor, how='left', on='site_path_timestamp')['floor']
#     test_data.insert(0, 'floor_pred', test_floor_pred)

    oof_x, oof_y, oof_f = np.zeros(data.shape[0]), np.zeros(data.shape[0]), np.zeros(data.shape[0])
    preds_x, preds_y = 0, 0
    preds_f_arr = np.zeros((test_data.shape[0], N_SPLITS))

    group_kfold = GroupKFold(n_splits=N_SPLITS)
    for fold, (trn_idx, val_idx) in enumerate(group_kfold.split(data.iloc[:, :-4], data.iloc[:, -4], data.iloc[:, -1])):
#     kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
#     for fold, (trn_idx, val_idx) in enumerate(kf.split(data.iloc[:, :-4])):
        X_train = data.iloc[trn_idx, :-4]
        y_trainx = data.iloc[trn_idx, -4]
        y_trainy = data.iloc[trn_idx, -3]
        y_trainf = data.iloc[trn_idx, -2]

        X_valid = data.iloc[val_idx, :-4]
        y_validx = data.iloc[val_idx, -4]
        y_validy = data.iloc[val_idx, -3]
        y_validf = data.iloc[val_idx, -2]

        modelx = lgb.LGBMRegressor(**lgb_params)
        with timer("fit X"):
            modelx.fit(X_train, y_trainx,
                       eval_set=[(X_train, y_trainx), (X_valid, y_validx)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=200
                       )

        modely = lgb.LGBMRegressor(**lgb_params)
        with timer("fit Y"):
            modely.fit(X_train, y_trainy,
                       eval_set=[(X_train, y_trainy), (X_valid, y_validy)],
                       eval_metric='rmse',
                       verbose=200,
                       early_stopping_rounds=200,
                       )

        modelf = lgb.LGBMClassifier(**lgb_f_params)
        with timer("fit F"):
            modelf.fit(X_train, y_trainf,
                       eval_set=[(X_train, y_trainf), (X_valid, y_validf)],
                       eval_metric='multi_logloss',
                       verbose=200,
                       early_stopping_rounds=20,
                       )

        oof_x[val_idx] = modelx.predict(X_valid)
        oof_y[val_idx] = modely.predict(X_valid)
#         oof_f[val_idx] = floor_pred[val_idx]
        oof_f[val_idx] = modelf.predict(X_valid).astype(int)

        preds_x += modelx.predict(test_data.iloc[:, :-1]) / N_SPLITS
        preds_y += modely.predict(test_data.iloc[:, :-1]) / N_SPLITS
#         preds_f_arr[:, fold] = test_floor_pred
        preds_f_arr[:, fold] = modelf.predict(test_data.iloc[:, :-1]).astype(int)

        score = comp_metric(oof_x[val_idx], oof_y[val_idx], oof_f[val_idx],
                            y_validx.to_numpy(), y_validy.to_numpy(), y_validf.to_numpy())
        print(f"fold {fold}: mean position error {score}")
        score_df = score_log(score_df, n_files, os.path.basename(file), data.shape, fold, SEED, score)

    print("*+"*40)
    print(f"file #{n_files}, shape={data.shape}, name={os.path.basename(file)}")
    score = comp_metric(oof_x, oof_y, oof_f,
                        data.iloc[:, -4].to_numpy(), data.iloc[:, -3].to_numpy(), data.iloc[:, -2].to_numpy())
    oof.append(score)
    print(f"mean position error {score}")
    print("*+"*40)
    score_df = score_log(score_df, n_files, os.path.basename(file), data.shape, 999, SEED, score)

    preds_f_mode = stats.mode(preds_f_arr, axis=1)
    preds_f = preds_f_mode[0].astype(int).reshape(-1)
    test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
    test_preds.columns = subm.columns
    test_preds.index = test_data["site_path_timestamp"]
    test_preds["floor"] = test_preds["floor"].astype(int)
    predictions.append(test_preds)

    oof_df = pd.DataFrame(np.array([oof_x, oof_y, oof_f, data['path'].values]).T, columns=['x', 'y', 'floor', 'path'])
    f_name = file.split('/')[-1].split('_')[0]
    oof_df.to_csv(str(OUTPUT_DIR) + f'/oof_{f_name}.csv', index=False)
    
#     break

[1.2GB(+0.2GB): 7.385sec] fit X


Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 1.47313	valid_1's rmse: 6.60525
[400]	training's rmse: 0.983684	valid_1's rmse: 6.5688
[600]	training's rmse: 0.758322	valid_1's rmse: 6.56047
[800]	training's rmse: 0.626591	valid_1's rmse: 6.53629
Early stopping, best iteration is:
[766]	training's rmse: 0.644713	valid_1's rmse: 6.5305


[1.2GB(+0.0GB): 13.800sec] fit Y


Training until validation scores don't improve for 20 rounds
[200]	training's multi_logloss: 0.0291422	valid_1's multi_logloss: 0.0600333
[400]	training's multi_logloss: 0.0286354	valid_1's multi_logloss: 0.0591984
[600]	training's multi_logloss: 0.0282437	valid_1's multi_logloss: 0.058518
Early stopping, best iteration is:
[586]	training's multi_logloss: 0.0282456	valid_1's multi_logloss: 0.0585173


[1.2GB(+0.0GB): 11.577sec] fit F


...

Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 1.1704	valid_1's rmse: 4.68939
[400]	training's rmse: 0.841563	valid_1's rmse: 4.66601
[600]	training's rmse: 0.689709	valid_1's rmse: 4.66215
[800]	training's rmse: 0.60259	valid_1's rmse: 4.65482
[1000]	training's rmse: 0.543619	valid_1's rmse: 4.66072
Early stopping, best iteration is:
[810]	training's rmse: 0.598773	valid_1's rmse: 4.65269


[1.7GB(+0.0GB): 6.438sec] fit Y


Training until validation scores don't improve for 20 rounds
[200]	training's multi_logloss: 0.0165825	valid_1's multi_logloss: 0.0419005
Early stopping, best iteration is:
[181]	training's multi_logloss: 0.0165825	valid_1's multi_logloss: 0.0419005
pos_loss = 5.315218967079758, floor_loss = 0.02498611882287618
fold 3: mean position error 5.340205085902634


[1.7GB(+0.0GB): 1.940sec] fit F
[1.7GB(+0.0GB): 8.613sec] fit X


Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 1.28681	valid_1's rmse: 3.96224
[400]	training's rmse: 0.933661	valid_1's rmse: 3.93989
[600]	training's rmse: 0.772923	valid_1's rmse: 3.9298


In [ ]:
# ------------------------------------------------------------------------------
# Submit the result
# ------------------------------------------------------------------------------
all_preds = pd.concat(predictions)
all_preds = all_preds.reindex(subm.index)
all_preds.to_csv(str(OUTPUT_DIR) + f'/sub_{EXP_NUM}.csv')

In [ ]:
all_preds

## Calc oof score

In [ ]:
oof_df_path = sorted(glob.glob(str(OUTPUT_DIR / 'oof*.csv')))
len(oof_df_path)

In [ ]:
oof_df = []
for path in oof_df_path:
    oof_df.append(pd.read_csv(path))
oof_df = pd.concat(oof_df)
oof_df

In [ ]:
train_csv_path = sorted(glob.glob(str(WIFI_DIR / '*train.csv')))
len(train_csv_path)

In [ ]:
train_df = []
for path in train_csv_path:
    train_df.append(pd.read_csv(path)[['x', 'y', 'f', 'path']])
train_df = pd.concat(train_df)
train_df

In [ ]:
oof_score = comp_metric(oof_df['x'], oof_df['y'], oof_df['floor'],
                            train_df['x'], train_df['y'], train_df['f'])
oof_score

In [ ]:
if IS_SAVE == True:
    mlflow.log_artifact(str(OUTPUT_DIR) + f'/sub_{EXP_NUM}.csv')
    mlflow.log_metric("oof_score", oof_score)
    mlflow.end_run()